In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import math

In [2]:
# features
outlook = ['sunny', 'sunny', 'overcast', 'rain', 'rain', 'rain', 'overcast', 'sunny', 'sunny', 'rain',
           'sunny', 'overcast', 'overcast', 'rain']
temperature = ['hot', 'hot', 'hot', 'mild', 'cool', 'cool', 'cool', 'mild', 'cool', 'mild',
               'mild', 'mild', 'hot', 'mild']
humidity = ['high', 'high', 'high', 'high', 'normal', 'normal', 'normal', 'high', 'normal', 'normal',
           'normal', 'high', 'normal', 'high']
wind = ['weak', 'strong', 'weak', 'weak', 'weak', 'strong', 'strong', 'weak', 'weak', 'weak', 'strong',
       'strong', 'weak', 'strong']
play = ['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no']

In [3]:
print(len(outlook))
print(len(temperature))
print(len(humidity))
print(len(wind))
print(len(play))

14
14
14
14
14


In [4]:
data = pd.DataFrame({'outlook': outlook, 'temperature': temperature, 'humidity': humidity,
                    'wind': wind, 'play': play})
data

,outlook,temperature,humidity,wind,play
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rain,mild,high,weak,yes
4,rain,cool,normal,weak,yes
5,rain,cool,normal,strong,no
6,overcast,cool,normal,strong,yes
7,sunny,mild,high,weak,no
8,sunny,cool,normal,weak,yes
9,rain,mild,normal,weak,yes


In [5]:
# utilities

def unique_values(target): # will return a dictionary consisting unique states and its total occurances of the target varible 
    record = dict(target.value_counts())
    return record


def entropy(feature): # returns the entropy(randomness) of the feature
    target_record = unique_values(feature)
    total_instance = len(feature)
    entropy_value = 0.0
    for i in target_record.values():
        entropy_value -= (i/total_instance) * math.log2(i/total_instance)
    return entropy_value

def info_gain(X, target, curr_feature): # returns the information gain of the current feature
    dataset_entropy = entropy(target)   # entropy of the whole dataset
    feature_value = set(X[curr_feature]) # gets unique values/states of the current feature
    total_instance = len(target)
    curr_feature_entropy = 0.0
    # formula can be found in the internet/books
    for value in feature_value:
        value_bool = X[curr_feature] == value
        value_entropy = entropy(pd.Series(target[value_bool]))
        value_weight = sum(value_bool) / total_instance
        curr_feature_entropy += value_weight * value_entropy
    information_gain = dataset_entropy - curr_feature_entropy
    return information_gain

def best_feature(X, target, features): # returns the feature that has the highest information gain
    best_feature = None
    highest_info_gain = -1
    for feature in features:
        gain = info_gain(X, target, feature)
        if gain > highest_info_gain:
            highest_info_gain = gain
            best_feature = feature
    return best_feature

def decisionTree(X, target, features): # building the decision tree recursively
    # if there is only one class then just return it
    if(len(set(target)) == 1):
        return target.iloc[0]
    # if there is no feature left to split then return the most frequent class
    if(len(features) == 0):
        return max(target)
    
    # gtting the feature with highest infomation gain
    feature_to_split = best_feature(X, target, features)
    rest_of_features = [feature for feature in features if feature != feature_to_split]
    
    tree = {feature_to_split : {}}
    
    for value in set(X[feature_to_split]):
        value_bool = X[feature_to_split] == value
        X_subset = X[value_bool]
        target_subset = target[value_bool]
        tree[feature_to_split][value] = decisionTree(X_subset, target_subset, rest_of_features)
        
    return tree

In [20]:
def predict(tree, instance): # returns the predicted value for the instance
    feature = list(tree.keys())[0] # feature at root node
    subtree = tree[feature] # will get the subtree under the root node
    value = instance[feature]  # will get the value of position at the particular instance and feature
    if value in subtree:
        prediction = subtree[value]
        if isinstance(prediction, dict): # if prediction is a dictionary type that means there a subtree under it
            return predict(prediction, instance) # recursive call
        else:  # no more subtree under it
            return prediction
    else: # value doesn't exist in the subtree and return the majority state of the parent node
        return max(subtree.values())

In [8]:
X = data.drop(columns = ['play'])
target = data['play']
features = list(X.columns)

In [9]:
tree = decisionTree(X, target, features)

In [10]:
print("Decision Tree:", tree)

Decision Tree: {'outlook': {'rain': {'wind': {'weak': 'yes', 'strong': 'no'}}, 'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}, 'overcast': 'yes'}}


### To visualize the Tree take a llok at the following code

In [16]:
def convert_tree_to_dot(tree, feature_names):
    dot = "digraph Tree {\nnode [shape=box, style=\"filled\", color=\"black\"] ;\n"
    queue = deque([(tree, "Root")])
    while queue:
        node, label = queue.popleft()
        if isinstance(node, dict):
            for value, subtree in node.items():
                if isinstance(subtree, dict):
                    queue.append((subtree, "{}_{}".format(label, value)))
                    dot += '"{}" -> "{}_{}" [label="{}"] ;\n'.format(label, label, value, value)
                else:
                    if subtree == 'yes':
                        dot += '"{}" [label="yes", shape=ellipse, fillcolor="#9ece9a"] ;\n'.format("{}_{}".format(label, value))
                    else:
                        dot += '"{}" [label="no", shape=ellipse, fillcolor="#ffcccb"] ;\n'.format("{}_{}".format(label, value))
                    dot += '"{}" -> "{}_{}" [label="{}"] ;\n'.format(label, label, value, value)
        else:
            dot += '"{}" [label="{}"] ;\n'.format(label, node)
    dot += "}"
    return dot

# Convert the decision tree to DOT format
dot_data = convert_tree_to_dot(tree, features)

# Visualize the decision tree
graph = graphviz.Source(dot_data)
graph.render("decision_tree", format='png', cleanup=True)
graph.view()


'decision_tree.pdf'

### Making predictions on test set

In [22]:
test_outlook = ['sunny', 'overcast', 'rain']
test_temperature = ['hot', 'mild', 'cool']
test_humidity = ['high', 'normal', 'high']
test_wind = ['weak', 'strong', 'weak']

test_data = pd.DataFrame({'outlook': test_outlook, 'temperature': test_temperature, 'humidity': test_humidity,
                    'wind': test_wind})
predictions = []
for i in range(len(test_data)):
    instance = test_data.iloc[i]
    predictions.append(predict(tree, instance))
test_data['play_predictions'] = predictions
test_data

,outlook,temperature,humidity,wind,play_predictions
0,sunny,hot,high,weak,no
1,overcast,mild,normal,strong,yes
2,rain,cool,high,weak,yes
